# How to?

In [ ]:
import numpy as np

size = 10
x1, x2 = np.random.rand(2, size)
x1 + x2

In [ ]:
import cupy as cp

size = 10
x1, x2 = cp.random.rand(2, size)
x1 + x2

In [ ]:
use_gpu = True

if use_gpu:
    import cupy as xp
else:
    import numpy as xp

# First look at performance

In [ ]:
from timeit import timeit
from pandas import DataFrame, Series

iters = 100
times = DataFrame(index=Series([2**i for i in range(24)], name="Size"))

for size in times.index:
    x1, x2 = np.random.rand(2, size)
    times.at[size, "Numpy"] = timeit(lambda: x1 + x2, number=iters) / iters

    x1, x2 = cp.random.rand(2, size)
    times.at[size, "Cupy"] = timeit(lambda: x1 + x2, number=iters) / iters

times.plot(logx=True, logy=True, ylabel="Time [s]", figsize=(8, 5))

# A deeper look into performance

*Note:* skip next cell (too advanced)

In [ ]:
# Some complex functions to display results

from IPython.core.display import display, HTML
import matplotlib as mpl
import base64
from io import BytesIO


def show(data, ylabel, filename, hline=None, show=True):
    "A way-too-complicated function for displaying results"

    backend_ = mpl.get_backend()
    mpl.use("Agg")  # Prevent showing plot

    ax = data.plot(logx=True, logy=True, ylabel=ylabel, figsize=(8, 5))
    if hline:
        ax.axhline(hline, ls="--", color="black")

    fig = filename + ".pdf"
    ax.figure.savefig(fig, bbox_inches="tight")  # Save figure

    csv = filename + ".txt"
    data.to_csv(csv)  # Save data to csv format

    if show:
        tmpfile = BytesIO()
        ax.figure.savefig(tmpfile, format="png", bbox_inches="tight")
        encoded = base64.b64encode(tmpfile.getvalue()).decode("utf-8")
        display(
            HTML(
                f"<h2>{ylabel}</h2><img src='data:image/png;base64,{encoded}'>"
                + data.transpose().to_html()
            )
        )

    mpl.use(backend_)  # Restore showing plot


def show_times(times, basename="", bandwidth=900, datasize=3 * 8):
    """
    This function displays some derived quantities from the provided timings.

    Namely:
    - the times themselves
    - the speedup compared to the first entry
    - the performance measured in GFLOP/s
    - the bandwidth measured in GB/s
    """

    # Timings
    show(times, "Time [s]", basename + "time")

    # Speedup
    key = next(iter(times))
    speedup = times.apply(lambda x: times[key] / x)
    show(speedup, f"Speed-up vs {key}", basename + "speedup")

    # Perfomance
    perf = times.apply(lambda x: np.array(times.index) / x / 1e9)
    show(perf, "Performance [GFLOP/s]", basename + "perf")

    # Bandwidth
    band = times.apply(lambda x: np.array(times.index) * datasize / x / 1e9)
    show(band, "Bandwidth [GB/s]", basename + "band", hline=bandwidth)

In [ ]:
show_times(times)

# Are the results correct??

# NO!!

because GPU timings are not measured and kernels are run asynchronously

# Solution?

We will use benchmark tool from cupy that synchronizes before timing

In [ ]:
from cupyx.profiler import benchmark

iters = 100
times = DataFrame(index=Series([2**i for i in range(25)], name="Size"))


def timeit_gpu(fnc, number=100):
    bench = benchmark(fnc, n_repeat=iters)
    cpu = bench.cpu_times.sum()
    gpu = bench.gpu_times.sum()
    return gpu  # + cpu


for size in times.index:
    x1, x2 = np.random.rand(2, size)
    times.at[size, "Numpy"] = timeit(lambda: x1 + x2, number=iters) / iters

    x1, x2 = cp.random.rand(2, size)
    times.at[size, "Cupy"] = timeit_gpu(lambda: x1 + x2, number=iters) / iters

show_times(times)

# Time for the first exercise

Look at exercise_1...

# Let's speed-up the kernel

Cupy has three kind of user-defined kernels

For more information see: https://docs.cupy.dev/en/stable/user_guide/kernel.html

## Element-wise kernel

In [ ]:
squared_diff = cp.ElementwiseKernel(
    "T x, T y",  # input params
    "T z",  # output params
    """
   T diff = x-y;
   z = diff*diff;
   """,  # element-wise kernel
    "squared_diff",  # kernel name
)

x, y = cp.random.rand(2, 10)
z = squared_diff(x, y)
z

## Reduction kernel

In [ ]:
l2norm = cp.ReductionKernel(
    "T x",  # input params
    "T y",  # output params
    "x * x",  # element-wise
    "a + b",  # reduce: fnc(a,b)
    "y = sqrt(a)",  # post-reduction fnc(a)
    "0",  # starting value
    "l2norm",  # kernel name
)

x = cp.random.rand(2, 10)
y = l2norm(x, axis=1)
y

## Raw, C-style kernel

In [ ]:
add_kernel = cp.RawKernel(
    """
        extern "C" __global__
        void raw_add(const float* x1, const float* x2, float* y) {
            int tid = blockDim.x * blockIdx.x + threadIdx.x;
            y[tid] = x1[tid] + x2[tid];
        }
    """,  # c-style kernel
    "raw_add",  # kernel name
)

x1, x2 = cp.random.rand(2, 5, 5)
y = cp.zeros((5, 5), dtype=cp.float32)
add_kernel((5,), (5,), (x1, x2, y))  # grid, block and arguments
y

# And now let's measure the gain in perfomance!

In [ ]:
iters = 100
times = DataFrame(index=Series([2**i for i in range(25)], name="Size"))

fnc = lambda: (x1 - x2) * (x1 - x2)
fnc2 = lambda: squared_diff(x1, x2)

for size in times.index:
    x1, x2 = np.random.rand(2, size)
    times.at[size, "Numpy"] = timeit(fnc, number=iters) / iters

    x1, x2 = cp.random.rand(2, size)
    times.at[size, "Cupy, inline"] = timeit_gpu(fnc, number=iters) / iters
    times.at[size, "Cupy, kernel"] = timeit_gpu(fnc2, number=iters) / iters

show_times(times)

# Time for the second exercise

Look at exercise_2...

# Rotation of coordinates (example from yesterday)

$$ x' = cos(\theta) x - sin(\theta) y + s_x $$
$$ y' = sin(\theta) x + cos(\theta) y + s_y $$

In [ ]:
def rot_shift(x, y, sx, sy, theta):
    xp = cp.get_array_module(x)
    cos = xp.cos(theta)
    sin = xp.sin(theta)
    x2 = cos * x - sin * y + sx
    y2 = sin * x + cos * y + sy
    return x2, y2

In [ ]:
iters = 1
times = DataFrame(index=Series([2**i for i in range(25)], name="Size"))

fnc = lambda: rot_shift(x, y, sx, sy, theta)

for size in times.index:
    x, y, sx, sy, theta = np.random.rand(5, size)
    times.at[size, "Numpy"] = timeit(fnc, number=iters) / iters

    x, y, sx, sy, theta = cp.random.rand(5, size)
    times.at[size, "Cupy"] = timeit_gpu(fnc, number=iters) / iters

show_times(times, datasize=7 * 8)

# Time for the third exercise

Look at exercise_3...

# Solution to the exercise

In [ ]:
def rot_shift(x, y, sx, sy, theta):
    xp = cp.get_array_module(x)
    cos = xp.cos(theta)
    sin = xp.sin(theta)
    x2 = cos * x - sin * y + sx
    y2 = sin * x + cos * y + sy
    return x2, y2


rot_shift_k = cp.ElementwiseKernel(
    "T x, T y, T sx, T sy, T theta",  # input params
    "T x2, T y2",  # output params
    """
    T cs = cos(theta);
    T sn = sin(theta);
    x2 = cs*x-sn*y+sx;
    y2 = sn*x+cs*y+sy;
   """,  # element-wise kernel
    "rot_shift_k",  # kernel name
)

In [ ]:
fnc = lambda: rot_shift_k(x, y, sx, sy, theta)

for size in times.index:
    x, y, sx, sy, theta = cp.random.rand(5, size)
    times.at[size, "Cupy, kernel"] = timeit_gpu(fnc, number=iters) / iters

show_times(times, datasize=7 * 8)

# Numba

In [ ]:
from numba import jit


def rot_shift_loop(coord, shift, theta):
    out = np.empty_like(coord)
    for i in range(coord.shape[0]):
        cos = np.cos(theta[i])
        sin = np.sin(theta[i])
        out[i, 0] = cos * coord[i, 0] - sin * coord[i, 1] + shift[i, 0]
        out[i, 1] = sin * coord[i, 0] + cos * coord[i, 1] + shift[i, 1]
    return out


rot_shift_numba = jit(rot_shift_loop)

In [ ]:
fnc = lambda: rot_shift_numba(coord, shift, theta)

for size in times.index:
    coord = np.random.rand(size, 2)
    shift = np.random.rand(size, 2)
    theta = np.random.rand(size)
    times.at[size, "Numba, cpu"] = timeit(fnc, number=iters) / iters

show_times(times, datasize=7 * 8)

In [ ]:
from numba import cuda
import math


@cuda.jit
def rot_shift_cuda(coord, shift, theta, out):
    i = cuda.grid(1)
    if i > len(theta):
        return
    cs = math.cos(theta[i])
    sn = math.sin(theta[i])
    out[i, 0] = cs * coord[i, 0] - sn * coord[i, 1] + shift[i, 0]
    out[i, 1] = sn * coord[i, 0] + cs * coord[i, 1] + shift[i, 1]


nthr = 256
fnc = lambda: rot_shift_cuda[math.ceil(size / nthr), nthr](coord, shift, theta, out)

for size in times.index:
    coord = cp.random.rand(size, 2)
    shift = cp.random.rand(size, 2)
    theta = cp.random.rand(size)
    out = cp.empty_like(coord)
    fnc()
    times.at[size, "Numba, cuda"] = timeit_gpu(fnc, number=iters) / iters

show_times(times, datasize=7 * 8)